# PyCity Schools Analysis



In [1]:
# Dependencies and Setup
import pandas as pd
from pathlib import Path

# File to Load (Remember to Change These)
school_data_to_load = Path("Resources/schools_complete.csv")
student_data_to_load = Path("Resources/students_complete.csv")

# Read School and Student Data File and store into Pandas DataFrames
school_data = pd.read_csv(school_data_to_load)
student_data = pd.read_csv(student_data_to_load)

# Combine the data into a single dataset.
school_data_complete = pd.merge(student_data, school_data, how="left", on=["school_name", "school_name"])
school_data_complete.head()

,Student ID,student_name,gender,grade,school_name,reading_score,math_score,School ID,type,size,budget
0,0,Paul Bradley,M,9th,Huang High School,66,79,0,District,2917,1910635
1,1,Victor Smith,M,12th,Huang High School,94,61,0,District,2917,1910635
2,2,Kevin Rodriguez,M,12th,Huang High School,90,60,0,District,2917,1910635
3,3,Dr. Richard Scott,M,12th,Huang High School,67,58,0,District,2917,1910635
4,4,Bonnie Ray,F,9th,Huang High School,97,84,0,District,2917,1910635


## District Summary

In [2]:
# Calculate the total number of unique schools
school_count =(school_data_complete ['school_name']).nunique()
school_count

15

In [3]:
# Calculate the total number of students
student_count =(school_data_complete['Student ID']).nunique()
student_count
# can use .nunique or .count 

39170

In [4]:
# Calculate the total budget
total_budget =school_data_complete.drop_duplicates(subset=['school_name'])['budget'].sum()
total_budget

24649428

In [5]:
# Calculate the average (mean) math score
average_math_score = (school_data_complete['math_score']).mean()
round(average_math_score,2)

78.99

In [6]:
# Calculate the average (mean) reading score
average_reading_score = (school_data_complete['reading_score']).mean()
round(average_reading_score,2)

81.88

In [7]:
# Use the following to calculate the percentage of students who passed math (math scores greather than or equal to 70)
passing_math_count = school_data_complete[(school_data_complete["math_score"] >= 70)].count()["student_name"]
passing_math_percentage = passing_math_count / student_count * 100
round(passing_math_percentage,2)

74.98

In [8]:
# Calculate the percentage of students who passed reading (hint: look at how the math percentage was calculated)
passing_reading_count = school_data_complete[(school_data_complete["reading_score"] >= 70)].count()["student_name"]
passing_reading_percentage = passing_reading_count / student_count*100
round(passing_reading_percentage,2)

85.81

In [9]:
# Use the following to calculate the percentage of students that passed math and reading
passing_math_reading_count = school_data_complete[(school_data_complete["math_score"] >= 70) & (school_data_complete["reading_score"] >= 70)
].count()["student_name"]
overall_passing_rate = passing_math_reading_count /  float(student_count) * 100
round(overall_passing_rate,2)

65.17

In [10]:
# Create a high-level snapshot of the district's key metrics in a DataFrame
district_summary = pd.DataFrame({ "Total School": [school_count],
                                  "Total Students": [student_count],
                                  "Total Budget": [total_budget],
                                  "Average Math Score": [average_math_score],
                                  "Average Reading Score": [average_reading_score],
                                 "% Passing Math": [passing_math_percentage],
                                 "% Passing Reading":[passing_reading_percentage],
                                 "% Overall Passing": [overall_passing_rate]})
# Formatting
district_summary["Total Students"] = district_summary["Total Students"].map("{:,}".format)
district_summary["Total Budget"] = district_summary["Total Budget"].map("${:,.2f}".format)
district_summary["Average Math Score"] = district_summary["Average Math Score"].map("{:,.2f}".format)
district_summary["Average Reading Score"] = district_summary["Average Reading Score"].map("{:,.2f}".format)
district_summary["% Passing Math"] = district_summary["% Passing Math"].map("{:,.2f}".format)
district_summary["% Passing Reading"] = district_summary["% Passing Reading"].map("{:,.2f}".format)
district_summary["% Overall Passing"] = district_summary["% Overall Passing"].map("{:,.2f}".format)

# Display the DataFrame
district_summary.style.set_properties(**{'text-align': 'center'}).set_table_styles([ dict(selector='th', props=[('text-align', 'center')] )])

,Total School,Total Students,Total Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
0,15,"39,170","$24,649,428.00",78.99,81.88,74.98,85.81,65.17


## School Summary

In [11]:
# Use the code provided to select the type per school from school_data
school_types = school_data.set_index(["school_name"])["type"]

In [12]:
# Calculate the total student count per school from school_data
per_school_counts = school_data_complete.groupby('school_name') ['student_name'].count()

In [13]:
# Calculate the total school budget and per capita spending per school from school_data
per_school_budget = school_data_complete.groupby('school_name') ['budget'].min()
per_school_capita = per_school_budget / per_school_counts

In [14]:
# Calculate the average test scores per school from school_data_complete
per_school_math = school_data_complete.groupby('school_name') ['math_score'].mean()
per_school_reading = school_data_complete.groupby('school_name') ['reading_score'].mean()

In [15]:
# Calculate the number of students per school with math scores of 70 or higher from school_data_complete
students_passing_math = school_data_complete[school_data_complete['math_score']>=70]
school_students_passing_math = students_passing_math.groupby('school_name')['Student ID'].count()

In [16]:
# Calculate the number of students per school with reading scores of 70 or higher from school_data_complete
students_passing_reading = school_data_complete[school_data_complete['reading_score']>=70]
school_students_passing_reading = students_passing_reading.groupby("school_name") ['Student ID'].count()
# school_students_passing_reading = students_passing_reading["school_name"].value_counts()

In [17]:
# Use the provided code to calculate the number of students per school that passed both math and reading with scores of 70 or higher
students_passing_math_and_reading = school_data_complete[
    (school_data_complete["reading_score"] >= 70) & (school_data_complete["math_score"] >= 70)
]
school_students_passing_math_and_reading = students_passing_math_and_reading.groupby(["school_name"]).size()

In [18]:
# Use the provided code to calculate the passing rates
per_school_passing_math = school_students_passing_math / per_school_counts * 100
per_school_passing_reading = school_students_passing_reading / per_school_counts * 100
overall_passing_rate = school_students_passing_math_and_reading / per_school_counts * 100

In [19]:
# Create a DataFrame called `per_school_summary` with columns for the calculations above.
per_school_summary = pd.DataFrame({"School Type":school_types,
                                  "Total Students":per_school_counts,
                                  "Total School Budget":per_school_budget,
                                  "Per Student Budget($)":per_school_capita,
                                  "Average Math Score":per_school_math,
                                  "Average Reading Score":per_school_reading,
                                  "% Passing Math":per_school_passing_math,
                                  "% Passing Reading":per_school_passing_reading,
                                  "% Overall Passing":overall_passing_rate})  



# Formatting
per_school_summary["Total School Budget"] = per_school_summary["Total School Budget"].map("${:,.2f}".format)
per_school_summary["Per Student Budget($)"] = per_school_summary["Per Student Budget($)"].map("{:,.2f}".format)
per_school_summary["Average Math Score"] = per_school_summary["Average Math Score"].map("{:,.2f}".format)
per_school_summary["Average Reading Score"] = per_school_summary["Average Reading Score"].map("{:,.2f}".format)
per_school_summary["% Passing Math"] = per_school_summary["% Passing Math"].map("{:,.2f}".format)
per_school_summary["% Passing Reading"] = per_school_summary["% Passing Reading"].map("{:,.2f}".format)
per_school_summary["% Overall Passing"] = per_school_summary["% Overall Passing"].map("{:,.2f}".format)


# Display the DataFrame
per_school_summary.style.set_properties(**{'text-align': 'center'}).set_table_styles([ dict(selector='th', props=[('text-align', 'center')] )])

,School Type,Total Students,Total School Budget,Per Student Budget($),Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
school_name,,,,,,,,,
Bailey High School,District,4976,"$3,124,928.00",628.00,77.05,81.03,66.68,81.93,54.64
Cabrera High School,Charter,1858,"$1,081,356.00",582.00,83.06,83.98,94.13,97.04,91.33
Figueroa High School,District,2949,"$1,884,411.00",639.00,76.71,81.16,65.99,80.74,53.20
Ford High School,District,2739,"$1,763,916.00",644.00,77.10,80.75,68.31,79.30,54.29
Griffin High School,Charter,1468,"$917,500.00",625.00,83.35,83.82,93.39,97.14,90.60
Hernandez High School,District,4635,"$3,022,020.00",652.00,77.29,80.93,66.75,80.86,53.53
Holden High School,Charter,427,"$248,087.00",581.00,83.80,83.81,92.51,96.25,89.23
Huang High School,District,2917,"$1,910,635.00",655.00,76.63,81.18,65.68,81.32,53.51
Johnson High School,District,4761,"$3,094,650.00",650.00,77.07,80.97,66.06,81.22,53.54


## Highest-Performing Schools (by % Overall Passing)

In [20]:
# Sort the schools by `% Overall Passing` in descending order and display the top 5 rows.
top_schools =per_school_summary.sort_values('% Overall Passing', ascending=False)
top_schools.head(5)

,School Type,Total Students,Total School Budget,Per Student Budget($),Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
school_name,,,,,,,,,
Cabrera High School,Charter,1858,"$1,081,356.00",582.00,83.06,83.98,94.13,97.04,91.33
Thomas High School,Charter,1635,"$1,043,130.00",638.00,83.42,83.85,93.27,97.31,90.95
Griffin High School,Charter,1468,"$917,500.00",625.00,83.35,83.82,93.39,97.14,90.60
Wilson High School,Charter,2283,"$1,319,574.00",578.00,83.27,83.99,93.87,96.54,90.58
Pena High School,Charter,962,"$585,858.00",609.00,83.84,84.04,94.59,95.95,90.54


## Bottom Performing Schools (By % Overall Passing)

In [21]:
# Sort the schools by `% Overall Passing` in ascending order and display the top 5 rows.
bottom_schools = per_school_summary.sort_values('% Overall Passing')
bottom_schools.head(5)

,School Type,Total Students,Total School Budget,Per Student Budget($),Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
school_name,,,,,,,,,
Rodriguez High School,District,3999,"$2,547,363.00",637.00,76.84,80.74,66.37,80.22,52.99
Figueroa High School,District,2949,"$1,884,411.00",639.00,76.71,81.16,65.99,80.74,53.20
Huang High School,District,2917,"$1,910,635.00",655.00,76.63,81.18,65.68,81.32,53.51
Hernandez High School,District,4635,"$3,022,020.00",652.00,77.29,80.93,66.75,80.86,53.53
Johnson High School,District,4761,"$3,094,650.00",650.00,77.07,80.97,66.06,81.22,53.54


## Math Scores by Grade

In [22]:
# Use the code provided to separate the data by grade
ninth_graders = school_data_complete[(school_data_complete["grade"] == "9th")]
tenth_graders = school_data_complete[(school_data_complete["grade"] == "10th")]
eleventh_graders = school_data_complete[(school_data_complete["grade"] == "11th")]
twelfth_graders = school_data_complete[(school_data_complete["grade"] == "12th")]

# Group by `school_name` and take the mean of the `math_score` column for each.
ninth_grade_math_scores = ninth_graders.groupby('school_name') ['math_score'].mean()
tenth_grader_math_scores = tenth_graders.groupby('school_name') ['math_score'].mean()
eleventh_grader_math_scores = eleventh_graders.groupby('school_name') ['math_score'].mean()
twelfth_grader_math_scores = twelfth_graders.groupby('school_name') ['math_score'].mean()

# Combine each of the scores above into single DataFrame called `math_scores_by_grade`
math_scores_by_grade = pd.DataFrame({'9th':ninth_grade_math_scores,
                                     '10th': tenth_grader_math_scores,
                                     '11th': eleventh_grader_math_scores,
                                     '12th': twelfth_grader_math_scores})
# Formatting
math_scores_by_grade["9th"] = math_scores_by_grade["9th"].map("{:,.2f}".format)
math_scores_by_grade["10th"] = math_scores_by_grade["10th"].map("{:,.2f}".format)
math_scores_by_grade["11th"] = math_scores_by_grade["11th"].map("{:,.2f}".format)
math_scores_by_grade["12th"] = math_scores_by_grade["12th"].map("{:,.2f}".format)

# Minor data wrangling
math_scores_by_grade.index.name = None

# Display the DataFrame
math_scores_by_grade.style.set_properties(**{'text-align': 'center'}).set_table_styles([ dict(selector='th', props=[('text-align', 'center')] )])

,9th,10th,11th,12th
Bailey High School,77.08,77.00,77.52,76.49
Cabrera High School,83.09,83.15,82.77,83.28
Figueroa High School,76.40,76.54,76.88,77.15
Ford High School,77.36,77.67,76.92,76.18
Griffin High School,82.04,84.23,83.84,83.36
Hernandez High School,77.44,77.34,77.14,77.19
Holden High School,83.79,83.43,85.00,82.86
Huang High School,77.03,75.91,76.45,77.23
Johnson High School,77.19,76.69,77.49,76.86
Pena High School,83.63,83.37,84.33,84.12


## Reading Score by Grade 

In [23]:
# Use the code provided to separate the data by grade
ninth_graders = school_data_complete[(school_data_complete["grade"] == "9th")]
tenth_graders = school_data_complete[(school_data_complete["grade"] == "10th")]
eleventh_graders = school_data_complete[(school_data_complete["grade"] == "11th")]
twelfth_graders = school_data_complete[(school_data_complete["grade"] == "12th")]

# Group by `school_name` and take the mean of the the `reading_score` column for each.
ninth_grade_reading_scores = ninth_graders.groupby('school_name') ['reading_score'].mean()
tenth_grader_reading_scores = tenth_graders.groupby('school_name') ['reading_score'].mean()
eleventh_grader_reading_scores = eleventh_graders.groupby('school_name') ['reading_score'].mean()
twelfth_grader_reading_scores = twelfth_graders.groupby('school_name') ['reading_score'].mean()

# Combine each of the scores above into single DataFrame called `reading_scores_by_grade`
reading_scores_by_grade =pd.DataFrame ({'9th': ninth_grade_reading_scores,
                           '10th': tenth_grader_reading_scores,
                           '11th': eleventh_grader_reading_scores,
                           '12th': twelfth_grader_reading_scores})

# Formatting
reading_scores_by_grade["9th"] = reading_scores_by_grade["9th"].map("{:,.2f}".format)
reading_scores_by_grade["10th"] = reading_scores_by_grade["10th"].map("{:,.2f}".format)
reading_scores_by_grade["11th"] = reading_scores_by_grade["11th"].map("{:,.2f}".format)
reading_scores_by_grade["12th"] = reading_scores_by_grade["12th"].map("{:,.2f}".format)


# Minor data wrangling
# reading_scores_by_grade = reading_scores_by_grade[["9th", "10th", "11th", "12th"]]
# reading_scores_by_grade.index.name = None

# Display the DataFrame
reading_scores_by_grade.style.set_properties(**{'text-align': 'center'}).set_table_styles([ dict(selector='th', props=[('text-align', 'center')] )])


,9th,10th,11th,12th
school_name,,,,
Bailey High School,81.30,80.91,80.95,80.91
Cabrera High School,83.68,84.25,83.79,84.29
Figueroa High School,81.20,81.41,80.64,81.38
Ford High School,80.63,81.26,80.40,80.66
Griffin High School,83.37,83.71,84.29,84.01
Hernandez High School,80.87,80.66,81.40,80.86
Holden High School,83.68,83.32,83.82,84.70
Huang High School,81.29,81.51,81.42,80.31
Johnson High School,81.26,80.77,80.62,81.23


## Scores by School Spending

In [24]:
# Establish the bins
spending_bins = [0, 585, 630, 645, 680]
labels = ["<$585", "$585-630", "$630-645", "$645-680"]

In [25]:
# Create a copy of the school summary since it has the "Per Student Budget"
school_spending_df = per_school_summary.copy()

In [26]:
# Use `pd.cut` to categorize spending based on the bins.
school_spending_df["Per Student Budget($)"] = school_spending_df["Per Student Budget($)"].astype(float)
school_spending_df["Spending Ranges (Per Student)"] = pd.cut(school_spending_df["Per Student Budget($)"],bins=spending_bins, labels=labels)
school_spending_df

,School Type,Total Students,Total School Budget,Per Student Budget($),Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing,Spending Ranges (Per Student)
school_name,,,,,,,,,,
Bailey High School,District,4976,"$3,124,928.00",628.0,77.05,81.03,66.68,81.93,54.64,$585-630
Cabrera High School,Charter,1858,"$1,081,356.00",582.0,83.06,83.98,94.13,97.04,91.33,<$585
Figueroa High School,District,2949,"$1,884,411.00",639.0,76.71,81.16,65.99,80.74,53.20,$630-645
Ford High School,District,2739,"$1,763,916.00",644.0,77.10,80.75,68.31,79.30,54.29,$630-645
Griffin High School,Charter,1468,"$917,500.00",625.0,83.35,83.82,93.39,97.14,90.60,$585-630
Hernandez High School,District,4635,"$3,022,020.00",652.0,77.29,80.93,66.75,80.86,53.53,$645-680
Holden High School,Charter,427,"$248,087.00",581.0,83.80,83.81,92.51,96.25,89.23,<$585
Huang High School,District,2917,"$1,910,635.00",655.0,76.63,81.18,65.68,81.32,53.51,$645-680
Johnson High School,District,4761,"$3,094,650.00",650.0,77.07,80.97,66.06,81.22,53.54,$645-680


In [27]:
#  Calculate averages for the desired columns.
school_spending_df["Average Math Score"] = school_spending_df["Average Math Score"].astype(float)
school_spending_df["Average Reading Score"] = school_spending_df["Average Reading Score"].astype(float)
school_spending_df["% Passing Math"] = school_spending_df["% Passing Math"].astype(float)
school_spending_df["% Passing Reading"] = school_spending_df["% Passing Reading"].astype(float)
school_spending_df["% Overall Passing"] = school_spending_df["% Overall Passing"].astype(float)
spending_math_scores = school_spending_df.groupby(["Spending Ranges (Per Student)"])["Average Math Score"].mean()
spending_reading_scores = school_spending_df.groupby(["Spending Ranges (Per Student)"])["Average Reading Score"].mean()
spending_passing_math = school_spending_df.groupby(["Spending Ranges (Per Student)"])["% Passing Math"].mean()
spending_passing_reading = school_spending_df.groupby(["Spending Ranges (Per Student)"])["% Passing Reading"].mean()
overall_passing_spending = school_spending_df.groupby(["Spending Ranges (Per Student)"])["% Overall Passing"].mean()

C:\Users\Lopez\AppData\Local\Temp\ipykernel_17672\1667593902.py:7: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  spending_math_scores = school_spending_df.groupby(["Spending Ranges (Per Student)"])["Average Math Score"].mean()
C:\Users\Lopez\AppData\Local\Temp\ipykernel_17672\1667593902.py:8: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  spending_reading_scores = school_spending_df.groupby(["Spending Ranges (Per Student)"])["Average Reading Score"].mean()
C:\Users\Lopez\AppData\Local\Temp\ipykernel_17672\1667593902.py:9: FutureWarning: The default of observed=False is deprecated and will be changed to True in

In [28]:
# Assemble into DataFrame
spending_summary = pd.DataFrame({'Average Math Score':spending_math_scores.map("{:,.2f}".format),
                                 'Average Reading Score':spending_reading_scores.map("{:,.2f}".format),
                                 '% Passing Math':spending_passing_math.map("{:,.2f}".format),
                                 '% Passing Reading': spending_passing_reading.map("{:,.2f}".format),
                                 '% Overall Passing':overall_passing_spending.map("{:,.2f}".format)})

     	
   
# Display results
spending_summary.style.set_properties(**{'text-align': 'center'}).set_table_styles([ dict(selector='th', props=[('text-align', 'center')] )])

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
Spending Ranges (Per Student),,,,,
<$585,83.45,83.93,93.46,96.61,90.37
$585-630,81.90,83.16,87.13,92.72,81.42
$630-645,78.52,81.62,73.48,84.39,62.86
$645-680,77.00,81.03,66.16,81.13,53.53


## Scores by School Size

In [30]:
# Establish the bins.
size_bins = [0, 1000, 2000, 5000]
labels = ["Small (<1000)", "Medium (1000-2000)", "Large (2000-5000)"]

In [31]:
# Categorize the spending based on the bins
# Use `pd.cut` on the "Total Students" column of the `per_school_summary` DataFrame.
per_school_summary["School Size"] = pd.cut(per_school_summary['Total Students'],bins=size_bins, labels=labels)
per_school_summary

,School Type,Total Students,Total School Budget,Per Student Budget($),Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing,School Size
school_name,,,,,,,,,,
Bailey High School,District,4976,"$3,124,928.00",628.00,77.05,81.03,66.68,81.93,54.64,Large (2000-5000)
Cabrera High School,Charter,1858,"$1,081,356.00",582.00,83.06,83.98,94.13,97.04,91.33,Medium (1000-2000)
Figueroa High School,District,2949,"$1,884,411.00",639.00,76.71,81.16,65.99,80.74,53.20,Large (2000-5000)
Ford High School,District,2739,"$1,763,916.00",644.00,77.10,80.75,68.31,79.30,54.29,Large (2000-5000)
Griffin High School,Charter,1468,"$917,500.00",625.00,83.35,83.82,93.39,97.14,90.60,Medium (1000-2000)
Hernandez High School,District,4635,"$3,022,020.00",652.00,77.29,80.93,66.75,80.86,53.53,Large (2000-5000)
Holden High School,Charter,427,"$248,087.00",581.00,83.80,83.81,92.51,96.25,89.23,Small (<1000)
Huang High School,District,2917,"$1,910,635.00",655.00,76.63,81.18,65.68,81.32,53.51,Large (2000-5000)
Johnson High School,District,4761,"$3,094,650.00",650.00,77.07,80.97,66.06,81.22,53.54,Large (2000-5000)


In [38]:
# convert to obj to float
per_school_summary["Average Math Score"] = per_school_summary["Average Math Score"].astype(float)
per_school_summary["Average Reading Score"] = per_school_summary["Average Reading Score"].astype(float)
per_school_summary["% Passing Math"] = per_school_summary["% Passing Math"].astype(float)
per_school_summary["% Passing Reading"] = per_school_summary["% Passing Reading"].astype(float)
per_school_summary["% Overall Passing"] = per_school_summary["% Overall Passing"].astype(float)
# Calculate averages for the desired columns.
size_math_scores = per_school_summary.groupby(["School Size"])["Average Math Score"].mean()
size_reading_scores = per_school_summary.groupby(["School Size"])["Average Reading Score"].mean()
size_passing_math = per_school_summary.groupby(["School Size"])["% Passing Math"].mean()
size_passing_reading = per_school_summary.groupby(["School Size"])["% Passing Reading"].mean()
size_overall_passing = per_school_summary.groupby(["School Size"])["% Overall Passing"].mean()

C:\Users\Lopez\AppData\Local\Temp\ipykernel_17672\4177008062.py:8: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  size_math_scores = per_school_summary.groupby(["School Size"])["Average Math Score"].mean()
C:\Users\Lopez\AppData\Local\Temp\ipykernel_17672\4177008062.py:9: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  size_reading_scores = per_school_summary.groupby(["School Size"])["Average Reading Score"].mean()
C:\Users\Lopez\AppData\Local\Temp\ipykernel_17672\4177008062.py:10: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=

In [34]:
# Create a DataFrame called `size_summary` that breaks down school performance based on school size (small, medium, or large).
# Use the scores above to create a new DataFrame called `size_summary`
size_summary = pd.DataFrame({'Average Math Score':size_math_scores.map("{:,.2f}".format),
                             'Average Reading Score': size_reading_scores.map("{:,.2f}".format),
                             '% Passing Math': size_passing_math.map("{:,.2f}".format),
                             '% Passing Reading':size_passing_reading.map("{:,.2f}".format),
                             '% Overall Passing':size_overall_passing.map("{:,.2f}".format)})

# Display results
size_summary.style.set_properties(**{'text-align': 'center'}).set_table_styles([ dict(selector='th', props=[('text-align', 'center')] )])

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
School Size,,,,,
Small (<1000),83.82,83.93,93.55,96.10,89.89
Medium (1000-2000),83.37,83.87,93.60,96.79,90.62
Large (2000-5000),77.75,81.34,69.96,82.77,58.28


## Scores by School Type

In [36]:
# Group the per_school_summary DataFrame by "School Type" and average the results.
average_math_score_by_type = per_school_summary.groupby(["School Type"])["Average Math Score"].mean()
average_reading_score_by_type = per_school_summary.groupby(["School Type"])["Average Reading Score"].mean()
average_percent_passing_math_by_type = per_school_summary.groupby(["School Type"])["% Passing Math"].mean()
average_percent_passing_reading_by_type = per_school_summary.groupby(["School Type"])["% Passing Reading"].mean()
average_percent_overall_passing_by_type = per_school_summary.groupby(["School Type"])["% Overall Passing"].mean()

In [37]:
# Assemble the new data by type into a DataFrame called `type_summary`
type_summary = pd.DataFrame({'Average Math Score':average_math_score_by_type.map("{:,.2f}".format),
                             'Average Reading Score':average_reading_score_by_type.map("{:,.2f}".format),
                             '% Passing Math':average_percent_passing_math_by_type.map("{:,.2f}".format),
                             '% Passing Reading':average_percent_passing_reading_by_type.map("{:,.2f}".format),
                             '% Overall Passing':average_percent_overall_passing_by_type.map("{:,.2f}".format)})


# Display results
type_summary.style.set_properties(**{'text-align': 'center'}).set_table_styles([ dict(selector='th', props=[('text-align', 'center')] )])

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
School Type,,,,,
Charter,83.47,83.90,93.62,96.59,90.43
District,76.96,80.97,66.55,80.80,53.67
